### scrape the table from wiki to dataframe

In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder

In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).text
soup = BeautifulSoup(html,'lxml')

In [3]:
table = soup.find('table', class_="wikitable sortable") #find the table source
postal_df = pd.DataFrame() #create an empty df
for i, row in enumerate(table.find_all('tr')): #<tr> row content </tr> 
    for j, col in enumerate(row.text.strip().split('\n')): #each column content is seperated by \n
        postal_df.loc[i,j] = col #populate postal_df

In [4]:
postal_df.columns = postal_df.iloc[0] #set the first row as header
postal_df.drop(0,axis=0,inplace=True) #drop the first row
postal_df.reset_index(inplace=True, drop=True) #reset the index

In [5]:
postal_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is Not assigned

In [6]:
postal_df.drop(postal_df[postal_df['Borough']=='Not assigned'].index, axis=0, inplace=True)
postal_df.reset_index(drop=True, inplace=True)
postal_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Assign neighbourhood the value of the borough if neighborhood Not assigned

In [7]:
for ind in postal_df[postal_df['Neighbourhood']=='Not assigned'].index:
    postal_df.loc[ind,'Neighbourhood']=postal_df.loc[ind,'Borough']
postal_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
postal_df.isnull().any() #check if any null values

0
Postcode         False
Borough          False
Neighbourhood    False
dtype: bool

### group neighbourhoods together in postcode

In [9]:
postcode_neighbourhood_dict={} #create dict with postcode:neighbourhood pairs
for postcode, group in postal_df.groupby('Postcode'):
    postcode_neighbourhood_dict[postcode] = group['Neighbourhood'].values

In [10]:
for i, row in postal_df.iterrows(): #re-assign the Neighbourhood with grouped neighbourhoods
    neighbourhoods = postcode_neighbourhood_dict[row['Postcode']]
    row['Neighbourhood'] = ', '.join(neighbourhoods)

In [11]:
postal_df.drop_duplicates(keep='first',inplace=True) #drop duplicate rows
postal_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


In [12]:
postal_df.shape

(103, 3)

### read the coordinates in the csv file

In [37]:
geo_coor = pd.read_csv('Geospatial_Coordinates.csv')

In [40]:
geo_coor.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [41]:
geo_coor.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
Toronto_df = pd.merge(postal_df,geo_coor,on='Postcode')
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
